In [1]:
import networkx as nx
import re

regex_valve = re.compile(r"Valve (?P<name>\w+) has flow rate=(?P<rate>\d+); tunnels? leads? to valves? (?P<valves>(\w+[, ]*)+)")

all_data = {}
with open("Day16.txt") as file:
    for line in file:
        if match := regex_valve.match(line.strip()):
            all_data[match["name"]] = (int(match["rate"]), match["valves"].split(", "))

graph = nx.DiGraph()
for valve, (rate, valves) in all_data.items():
    graph.add_node(valve, rate=rate) if rate else graph.add_node(valve)
for valve, (rate, valves) in all_data.items():
    for dest in valves:
        graph.add_edge(valve, dest, weight=1)

In [2]:
distances = {
    node: {
        dest: nx.shortest_path_length(graph, node, dest) 
        for dest in graph.nodes if node != dest
    } for node in graph.nodes
}

def get_valves(valve="AA", open_valves=None, time_left=30):
    open_valves = tuple(open_valves or [])
    for next_valve, data in graph.nodes.items():
        if not data:
            continue
        if next_valve not in open_valves and (distance := distances[valve][next_valve]) < time_left:
            yield from get_valves(next_valve, open_valves + (next_valve, ), time_left - distance - 1)
    yield open_valves
    
def get_pressure(valves, time_left=30):
    valve, pressure = "AA", 0
    for next_valve in valves:
        time_left -= distances[valve][next_valve] + 1
        pressure += graph.nodes[next_valve]["rate"] * time_left
        valve = next_valve
    return pressure

In [3]:
%%time
max(get_pressure(valves) for valves in get_valves())

CPU times: user 4.81 s, sys: 1.11 ms, total: 4.81 s
Wall time: 4.83 s


2183

In [4]:
%%time
from itertools import permutations

best_pressures = {}
for valves in get_valves(time_left=26):
    open_valves = frozenset(valves)
    best_pressure = best_pressures.get(open_valves, 0)
    best_pressures[open_valves] = max(best_pressure, get_pressure(valves, time_left=26))

max_pressure = 0
for (valves_1, pressure_1), (valves_2, pressure_2) in permutations(best_pressures.items(), 2):
    if valves_1 & valves_2:
        continue
    max_pressure = max(max_pressure, pressure_1 + pressure_2)
max_pressure

CPU times: user 10.4 s, sys: 4 ms, total: 10.4 s
Wall time: 10.5 s


2911